# Plant Classification

### Oumaima KHADIRA - Zaim ZIZI - Sarah ASSAF 

In our project we will give an overall view of the models and methods used to handle a plant classification problem of 153 classes, we will show the intermediate work we did in order to get the final and best accuracy.

#### 1. Data
   * Import libraries
   * Files path initialization
   * Data exploration
        * Data visualization
        * Problems with the datasets
        * Classes distribution
   * Data augumentation
   * Image preprocessing
   * Definition of dataset
   * Data loaders
   
#### 2. Modeling
   * Hyperparameters
        * Optimizer
        * Crieterion
        * Batch size
        * Learning rate
   * Pretrained models and results
        * Resnet18
        * Vgg19
        * Alexnet
        * Googlenet
        * Efficientnet b5
        * Efficientnet b0
   
#### 3. Training
   * Training loop
   * Training model
   * Generate output csv files
   
#### 4. Testing
   
#### 5. Conclusion

# 1. Data

### 1.1. Import libraries

In [ ]:
!pip install efficientnet_pytorch

In [ ]:
from IPython.display import display #We need the display function from IPython for Jupyter Notebook/Colab
from matplotlib.pyplot import imshow
from PIL import Image
from torchvision import  transforms
from efficientnet_pytorch import EfficientNet
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torchvision
import torch.nn as nn
import os
import tqdm.auto as tqdm
import time
import torch.nn.functional as F
import cv2
import glob
import matplotlib.pyplot as plt
import seaborn as sns

### 1.2. Files path initialization

In [ ]:
BASE_TRAIN_URL = "/kaggle/input/polytech-nice-data-science-course-2021/polytech/train"
BASE_TEST_URL = "/kaggle/input/polytech-nice-data-science-course-2021/polytech/test"

### 1.3. Data exploration

#### 1.3.1. Data visualization
We display 10 images randomly with their labels just to get an idea of the dataset

In [ ]:
SCOPE_LIMIT = 11

for i in range(1,SCOPE_LIMIT):
    
    train_path = BASE_TRAIN_URL + f"/{i}"
    ctr = 0
    for dirname, _, filenames in os.walk(train_path):
        
        for filename in filenames:
            if ctr >= 1:
                break
            print(f"This is an example of image type : {i}")
            pil_im = Image.open(os.path.join(dirname, filename), 'r').convert('RGB')
            display(pil_im)
            ctr = ctr + 1

![](![Screenshot (20).png](attachment:ae7bae4f-59af-41fa-9f41-9c3100613ad7.png))

#### 1.3.2. Problems with the datasets
After exploring the dataset we found various aspects that will negatively affect the performance:

1. There are images which does provide a clear idea about the plant in question. 
For example, we can notice in the images below that only a trunk of the tree is shown. 

In [ ]:
img=[Image.open('/kaggle/input/polytech-nice-data-science-course-2021/polytech/train/94/03aa4199e7a524b1347d6ff6a71624fb72e0a458.jpg'), 
     Image.open('/kaggle/input/polytech-nice-data-science-course-2021/polytech/train/105/31f865004049633b61df3728fe14cd644e6b26f7.jpg')]

display(*img)

![](![Screenshot (21).png](attachment:6f5b3641-82d6-4c6f-a49f-268056af15c9.png))

2. Some plants are dry, which gives a different visualization of the true plant.

In [ ]:
img=Image.open('/kaggle/input/polytech-nice-data-science-course-2021/polytech/train/13/05afa0142c4ee1e85cab3983f7917238862d1880.jpg')

display(img)

![](![Screenshot (22).png](attachment:815a85b5-b67d-4980-a3aa-bbfce223ff25.png))

3.There are some plants that have not yet flowered.

In [ ]:
img=Image.open('/kaggle/input/polytech-nice-data-science-course-2021/polytech/train/1/03e33a2552a32be78ce4de23a432ab6b221e6717.jpg')
display(img)

![](![Screenshot (23).png](attachment:85ddd9c8-0a57-4e1a-935e-8df82e2b6148.png))

#### 1.3.3. Classes distribution:

We are curious to see the distribution of each class of our dataset, so we can have an idea from where to begin our work.

Note that all images in train/ or test/ are of .jpg extension

In [ ]:
print(os.listdir("../input"))

In [ ]:
train_image_names = glob.glob('../input/polytech-nice-data-science-course-2021/polytech/train/*/*.jpg') 
print("Total number of training images: ", len(train_image_names))

#make train_image_names as serie object¶
train_image_names = pd.Series(train_image_names)

In [ ]:
# train_df: a dataframe with 2 field: Filename, ClassId
train_df = pd.DataFrame()

# generate Filename field
train_df['Filename'] = train_image_names.map(lambda img_name: img_name.split("/")[-1])

# generate ClassId field
train_df['ClassId'] = train_image_names.map(lambda img_name: int(img_name.split("/")[-2]))

class_id_distribution = train_df['ClassId'].value_counts()
class_id_distribution.head(10)

In [ ]:
plt.figure(figsize=(60,20))
plt.xticks(np.arange(154))
plt.bar(class_id_distribution.index, class_id_distribution.values, color=(0.8, 0.2, 0.4, 0.8))

![](![Screenshot (27).png](attachment:e8f7f41d-7b1c-4ad8-9898-b5e54432e487.png))

As we can see in the previous figures the Dataset is unbalanced since in the training dataset we can find classes with more than 6000 samples (i.e. class 11) and classes with less than 120 samples (i.e. class 1)

### 1.4. Data augmentation

When exploring our data we noticed that there are classes having a limited number of images, when others have much more examples, so we decided to use a data augmentation on the classes with fewer data examples so we can  increase the number of images trained by the network and in order to reduce the skews in data (one class having more data than the other). To do that we randomly resize, flip and crop the images so we can get many new versions of a single image and apply a different random transformation on each epoch.

In [ ]:
preprocess= transforms.Compose([transforms.Resize((256,256)),
                              transforms.CenterCrop(224),
                              transforms.ToTensor(),
                            transforms.RandomHorizontalFlip(),
                            transforms.RandomVerticalFlip(),
                            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])])

image_transforms = {
            'train':preprocess,
            'val': preprocess,
            'test':preprocess
            } 

### 1.5. Image Preprocessing

In [ ]:
def imshow_tensor(image, ax=None, title=None):
    """Imshow for Tensor."""

    if ax is None:
        fig, ax = plt.subplots()

    # Set the color channel as the third dimension
    image = image.numpy().transpose((1, 2, 0))

    # Reverse the preprocessing steps
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean

    # Clip the image pixel values
    image = np.clip(image, 0, 1)

    ax.imshow(image)
    plt.axis('off')

    return ax, image

In [ ]:
def imshow(image):
    """Display image"""
    plt.figure(figsize=(6, 6))
    plt.imshow(image)
    plt.axis('off')
    plt.show()


# Example image
x = Image.open('/kaggle/input/polytech-nice-data-science-course-2021/polytech/train/101/032677244d114016482145eae92ef64579fbb080.jpg')
np.array(x).shape
imshow(x)

![](![Screenshot (28).png](attachment:ee7ad0a3-cf07-46c4-aa61-934c3ae1f0a8.png))

In [ ]:
t = image_transforms['train']
plt.figure(figsize=(16, 16))

for i in range(16):
    ax = plt.subplot(4, 4, i + 1)
    _= imshow_tensor(t(x), ax=ax)

plt.tight_layout()

![](![Screenshot (29).png](attachment:c02d0687-1f5a-461c-87d8-d62844182686.png))

In [ ]:
Unfortunately, this technique did not work with us since it gives a greater accuracy when using the old dataset without the augmentation. This is maybe because the most tested classes are the ones trained the most by the network.
So we decided not to use it and keep the data without augmentation, since we are searching for the better accuracy
Instead of that, we implemented a new method where where we make just a few transformations on the images of our network using image transforms. The operations were to resize the images to 299 x 299 and normalize them by subtracting the mean and dividing by the standard deviation. 

In [ ]:
preprocess = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


### 1.6. Definition of our dataset

In [ ]:
class Dataset(torch.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, path_images=BASE_TRAIN_URL, transform=None, dataset_type='train'):
        'Initialization'
        
        # The attribute dataset is a dataframe with the data in csv defined by path_labels
        # The csv contains the name of images in the first column and their labels in the second
        
        self.label_start = 1
        self.label_end   = 153
        self.base_path_images = path_images
        self.transform = transform
        self.image_list = []
        self.dataset_type = dataset_type
        
        tmp_map = {}
        
        for dirname, _, filenames in os.walk(self.base_path_images):
            
            for filename in filenames:
                if '.jpg' in filename and '.comments' not in dirname:
                    if self.dataset_type == 'test':
                        label = os.path.join(dirname, filename).split('/')[-1].split('.')[0]
                    else:
                        label = int(os.path.join(dirname, filename).split('/')[-2])
                    full_filename = os.path.join(dirname, filename)
                    
                    if label not in tmp_map:
                        tmp_map[label] = []
                        
                    tmp_map[label].append(full_filename)
                    
        for l in sorted(list(tmp_map.keys())):
            if self.dataset_type == 'train':
                self.image_list += tmp_map[l][:int(0.8*len(tmp_map[l]))]
            if self.dataset_type == 'val':
                self.image_list += tmp_map[l][:int(0.8*len(tmp_map[l]))]
            if self.dataset_type == 'test':
                self.image_list += tmp_map[l]

        del tmp_map
        
  def __len__(self):
        'Denotes the total number of samples'
        return len(self.image_list)

  def __getitem__(self, index):
        'Generates one sample of data'

        # Load data and get label
        X = Image.open(self.image_list[index]).convert('RGB') # Open image at index and convert to RGB
        if self.dataset_type == 'test':
            y = self.image_list[index].split('/')[-1].split('.')[0]
        else:
            y = int(self.image_list[index].split('/')[-2]) - 1
        
        if(self.transform):
            X = self.transform(X) # Apply transformations on the image if defined

        return X, y

In [ ]:
train_set = Dataset(dataset_type = 'train', transform = preprocess)
val_set  = Dataset(dataset_type = 'val', transform = preprocess)

In [ ]:
test_set = Dataset(path_images = BASE_TEST_URL, dataset_type = 'test', transform = preprocess)

## 2. Modeling

For this project, we thought to try different neural network models on our plant dataset, and then choose the model that provides the best accuracy. We tried the models: resnet18, googlnet, alexnet, vgg19, efficientnet_b0, efficientnet_b5. We saw that the efficientnet_b0 model comes with the best acuuracy. In the following parts, we will show some results of some models we tried and our chosen settings for them.

### 2.1. Hyperparameters
#### 2.1.1. Optimizer
We compared two of the most popular optimizers, ADAM and SGD, we found that ADAM had given a better accuracy than SGD (training accuracy = 0.049 for efficientnet_b0 in the first epoch, while it gives 0.754 for the same model, same epoch, with ADAM), so we chose ADAM as the optimizer.
#### 2.1.2. Criterion
The loss crieterion chosen is the negative log likelihood in PyTorch.
#### 2.1.3. Batch size
First we tried all the models with a batch size = 32, then we had increased it to 64, which gives almost better accuracies, we also tried batch_size = 128, but our machines were not able to do it because of a memory issue, so we kept the batch size 64.
#### 2.1.4. Learning rate
Learning rate is very important for an optimizer,the final goal is to acheive the global minimum of the loss function.
We tried the learning rate of 0.01,0.001 and 0.0001 as the starter learning rate. For the first one the result is very unstable and often can't get a loss that low enough. The learning rate 0.001 and 0.0001 have almost the same performance, but 0.001 are faster than 0.0001,so we choose the 0.001 as the learning rate.

### 2.2. Pretrained models and results

#### 2.2.1. Resnet18

In [ ]:
'''def resNet():
    LEARNING_RATE = 0.001
    N_EPOCHS = 5
    
    # Define the neural network
    net = torchvision.models.resnet18(pretrained=True)
    set_parameter_requires_grad(net, True)
    num_ftrs = net.fc.in_features
    net.fc = nn.Linear(num_ftrs, 153)
    
    # Move model to the GPU
    net = net.cuda()
    
    # Define images transformations (pre processing)
    convert = torchvision.transforms.Compose([torchvision.transforms.Resize((299,299)), torchvision.transforms.ToTensor()])

    # Negative log likelihood loss
    criterion = nn.CrossEntropyLoss().cuda()

    # Stochastic Gradient Descent
    optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE)
    
    return net, convert, criterion, optimizer'''

By launching the resnet18 model, with learning rate 0.001, batch size 64, optimizer ADAM, we obtained the following results:

-epochs 0:
     Training accuracy: 0.5366598576046645 and Training loss: 0.12245032447465524
     
     Validation accuracy: 0.6700811137774428 and Validation loss: 0.0797195799488145
     
 -epoch1:
     Training accuracy: 0.6223945421732147 and Training loss: 0.0919386227157948
     
     Validation accuracy: 0.6836860163020844 and Validation loss: 0.07436497368758811
     
 -epoch2:
     Training accuracy: 0.6390337742696786 and Training loss: 0.08660303843129435
     
     Validation accuracy: 0.6951688713483926 and Validation loss: 0.07099304307477727

#### 2.2.2. VGG19

In [ ]:
'''def vgg19():
    N_EPOCHS = 20
    LEARNING_RATE = 0.001
    net = torchvision.models.vgg19(pretrained=True)

    # Move model to the GPU
    net = net.cuda()
    
    # Define images transformations (pre processing)
    convert = transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])


    # Negative log likelihood loss
    criterion = nn.CrossEntropyLoss().cuda()

    # Stochastic Gradient Descent
    optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE)
    
    return net, convert, criterion, optimizer'''

By launching the vgg19 model, with learning rate 0.001, batch size 64, optimizer ADAM, we obtained the following results:

-epochs 0:
     training accuracy: 0.0565218235357475755   and  training loss : 0.06833281091570115
     
     validation accuracy: 0.046080161830910495   and  validation loss     0.06843311045526965
     
 -epoch1:
     training accuracy: 0.05219840152311445   and  training loss : 0.068519341367136713217
     
     validation accuracy: 0.054677428951073914   and  validation loss     0.068398682803759571
     
 -epoch2:
     training accuracy: 0.05336850246911132   and  training loss : 0.0646130218859163
     
     validation accuracy: 0.054677428951073914  and  validation loss     0.068398682803759571

#### 2.2.3. Alexnet

In [ ]:
'''def alexNet():
    N_EPOCHS = 20
    LEARNING_RATE = 0.001
    net = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)

    # Move model to the GPU
    net = net.cuda()
    
    # Define images transformations (pre processing)
    convert = transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])


    # Negative log likelihood loss
    criterion = nn.CrossEntropyLoss().cuda()

    # Stochastic Gradient Descent
    optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE)
    
    return net, convert, criterion, optimizer'''

By launching the vgg19 model, with learning rate 0.001, batch size 64, optimizer ADAM, we obtained the following results:

-epochs 0:
     training accuracy: 0.05526247942407235   and   training loss : 0.0689339786274436
     
     validation accuracy: 0.054677428951073914   and   validation loss     0.06843120239974558  
     
 -epoch1:
     training accuracy: 0.052723955337841855   and   training loss : 0.06847859151213531
     
     validation accuracy: 0.054677428951073914   and   validation loss     0.068398682803759571
     
 -epoch2:
     training accuracy: 0.05349741189536521   and  training loss : 0.06846043126650082
     
     validation accuracy: 0.047379172203161256   and  validation loss     0.06838812926981457  

#### 2.2.4. Googlenet

In [ ]:
'''def googleNet():
    LEARNING_RATE = 0.001
    N_EPOCHS = 9

    net = torchvision.models.googlenet(pretrained=True)

    # Move model to the GPU
    net = net.cuda()
    
    # Define images transformations (pre processing)
    convert = transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])


    # Negative log likelihood loss
    criterion = nn.CrossEntropyLoss().cuda()

    # Stochastic Gradient Descent
    optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE)
    
    return net, convert, criterion, optimizer'''

By launching the googlenet model, with learning rate 0.001, batch size 64, optimizer ADAM, we obtained the following results:

-epochs 0:
     Training accuracy: 0.6685738651012435 and Training loss: 0.02038898156955875
     
     Validation accuracy: 0.7747853162247387 and Validation loss: 0.012348674944767131
     
 -epoch1:
     Training accuracy: 0.7839676338179006 and Training loss: 0.011803226653651466
     
     Validation accuracy: 0.8151637149713424 and Validation loss: 0.009688149456260503
     
 -epoch2:
     Training accuracy: 0.8197846220970589 and Training loss: 0.009557194838614082
     
     Validation accuracy: 0.8174146718759296 and Validation loss: 0.009569979449191622  

We also tried this model with the data augmentation maybe it will yields better results, but it gave a lower accuracy: 0.803862 for the training accuracy of the third epoch.


#### 2.2.5. Efficientnet b5

In [ ]:
'''def efficientNet():
    LEARNING_RATE = 0.001
    N_EPOCHS = 20

    net = EfficientNet.from_pretrained('efficientnet-b0')

    # Move model to the GPU
    net = net.cuda()
    
    # Define images transformations (pre processing)
    convert = transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])


    # Negative log likelihood loss
    criterion = nn.CrossEntropyLoss(reduction="mean").cuda()

    # Stochastic Gradient Descent
    optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE)
    
    return net, convert, criterion, optimizer'''

By launching the efficientnet_b0 model, with learning rate 0.001, batch size 64, optimizer ADAM, we obtained the following results:

-epochs 0:
     Training accuracy: 0.6719354262935565 and Training loss: 0.07986202374569976
     
     Validation accuracy: 0.740523091109318 and Validation loss: 0.00801569725346

#### 2.2.6. Efficientnet b0 

In [ ]:
def efficientNet():
    LEARNING_RATE = 0.001
    N_EPOCHS = 20

    net = EfficientNet.from_pretrained('efficientnet-b0')

    # Move model to the GPU
    net = net.cuda()
    
    # Define images transformations (pre processing)
    convert = transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])


    # Negative log likelihood loss
    criterion = nn.CrossEntropyLoss(reduction="mean").cuda()

    # Stochastic Gradient Descent
    optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE)
    
    return net, convert, criterion, optimizer

By launching the efficientnet_b0 model, with learning rate 0.001, batch size 64, optimizer ADAM, we obtained the following results:

-epochs 0:
     Training accuracy: 0.7541399758046923 and Training loss: 0.014889113296791838
     
     Validation accuracy: 0.830722091109216 and Validation loss: 0.009014848150113275
     
 -epoch1:
     Training accuracy: 0.8465581183190211 and Training loss: 0.008048727570332386
     
     Validation accuracy: 0.8671439620807965 and Validation loss: 0.006600792605357559
     
 -epoch2:
     Training accuracy: 0.8728258929456796 and Training loss: 0.006469210704176352
     
     Validation accuracy: 0.9054598100073379 and Validation loss: 0.00463159754480754  
     
     
We noticed that this model has given the best accuracy for the validation and the training so we applied it on 10 epochs, it gives the results for the epoch 10:

Training accuracy: 0.9420800031731551 and Training loss: 0.0026503791539075876

Validation accuracy: 0.9456597187791286 and Validation loss: 0.002409334887945289


Hence, we decided to apply it on 20 epochs for the final submission, hoping it will yields to better results, but again, we had a memory issue in our machines that we were not able to turn the model on more than 10 epochs, so we kept our final model the efficientnet_b0 with a training and validation accuracy of 94% after 10 epochs.

![](![Screenshot (24).png](attachment:12e068fc-4a48-401c-91e0-eaae397e46a4.png))

![](![Screenshot (25).png](attachment:6d54c888-bbed-4699-b8f7-5001c1c45538.png))

![](![Screenshot (26).png](attachment:60b60e90-45cb-46fb-bbe7-8785f0d342a2.png))

## 3. Training
For the training loop, we iterate using the train dataloader, at each time one batch in the model. Each time we send a batch of inputs through the model, we call it epoch, and we train our model for a precise number of epochs. 

At the end of the training loop, we record the loss and the accuracy for training and validation for each epoch in order to compare models and get the best results.

### 3.1. Training loop

In [ ]:
def train_and_eval(net, criterion, optimizer, train_dl, val_dl, hidden=False):
    '''
    Train and evaluate a neural network
    -- Inputs:
    net: neural network
    criterion: loss function
    optimizer: optimizer function
    train_dl: train dataset data loader
    val_dl: val dataset data loader
    '''
    
    epoch_loss, epoch_acc, epoch_val_loss, epoch_val_acc = [], [], [], []

    for e in range(N_EPOCHS):
        print("EPOCH:", e)

        ### TRAINING LOOP
        running_loss = 0
        running_accuracy = 0

        # Put the network in training mode
        net.train()

        for i, batch in enumerate(tqdm.tqdm(train_dl)):

            # Get a batch from the dataloader
            x = batch[0]
            labels = batch[1]

            # Move the batch to GPU
            x = x.cuda()
            labels = labels.cuda()

            # Compute the network output
            if hidden:
                y, hidden = net(x)
            else:
                y = net(x)

            # Compute the loss
            loss = criterion(y, labels)

            # Reset the gradients
            optimizer.zero_grad()

            # Compute the gradients
            loss.backward()

            # Apply one step of the descent algorithm to update the weights
            optimizer.step()

            # Compute some statistics
            with torch.no_grad(): # disables gradient calculations
                running_loss += loss.item()
                running_accuracy += (y.max(1)[1] == labels).sum().item()

        print("Training accuracy:", running_accuracy/float(len(train_set)),
              "Training loss:", running_loss/float(len(train_set)))

        epoch_loss.append(running_loss/len(train_set))
        epoch_acc.append(running_accuracy/len(train_set))

        ### VALIDATION LOOP
        # Put the network in validation mode
        net.eval()

        running_val_loss = 0
        running_val_accuracy = 0

        for i, batch in enumerate(tqdm.tqdm(val_dl)):

            with torch.no_grad(): # disables gradient calculations
                # Get a batch from the dataloader
                x = batch[0]
                labels = batch[1]

                # Move the batch to GPU
                x = x.cuda()
                labels = labels.cuda()

                # Compute the network output
                if hidden:
                    y, hidden = net(x)
                else:
                    y = net(x)

                # Compute the loss
                loss = criterion(y, labels)

                running_val_loss += loss.item()
                running_val_accuracy += (y.max(1)[1] == labels).sum().item()

        print("Validation accuracy:", running_val_accuracy/float(len(val_set)),
              "Validation loss:", running_val_loss/float(len(val_set)))

        epoch_val_loss.append(running_val_loss/len(val_set))
        epoch_val_acc.append(running_val_accuracy/len(val_set))

In [ ]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

### 3.2. Training model

In [ ]:
N_EPOCHS=20

In [ ]:
net, convert, criterion, optimizer = efficientNet()

### 3.3. Generate output csv file

In [ ]:
def test_network(network, test_dl, hidden=False):
    
    network.eval()

    test_files_labels = []
    outputs           = []

    for i, batch in enumerate(tqdm.tqdm(test_dl)):

        with torch.no_grad(): # disables gradient calculations
            # Get a batch from the dataloader
            x = batch[0]
            labels = batch[1]

            # Move the batch to GPU
            x = x.cuda()
            

            # Compute the network output
            if hidden:
                y, hidden = net(x)
            else:
                y = net(x)
            
            test_files_labels += labels
            outputs += (list(y.cpu().numpy().argmax(axis=1) + 1 ) )
            
            if (i % 100) == 0: 
                print(f"batch {i} Done")
            
    
    submission_df = pd.DataFrame({"image_name" : test_files_labels, "class" : outputs})
    submission_df['image_name'] = submission_df['image_name'].apply(lambda x : str(x)+'.jpg')
    submission_df.to_csv("submission_efficient.csv", index=False)
    
    print("File was saved as 'submission_efficient.csv'")
    

In [ ]:
train_and_eval(net, criterion, optimizer, train_dl, val_dl)

## 4. Testing

In [ ]:
test_network(net, test_dl)

In this part, we thought to make a confusion matrix, but then we decided not, for we have so much classes and the matrix would not be lisible to allow us compare the predicted and real classes of the images.

## 5. Conclusion
Now, let's review the process of our project.
​
First, we used the materials we have learned and many others online, to be able to classify the plant images the better way. 
​
To do that we tried different models from the PyTorch library suitable for image classification, and for that we learned so much. 
While exploring the data we found that it was too large, so we tried the data augmentation, but when we applied the efficientnet model on the training set, we didn't notice a great change in the accuracy, neither with the googlenet model, so we continued without the augmentation.
​
Then we have studied and tested the models function and the different parameter settings in each one.
​
We faced a real problem with the memory, so a model like Xception did not be able to run, same for a large number of epochs and larger batch size. We beleive that these parameters may increase more our accuracy but unfortunately we could not.
​
After we get the result, we found that balancing the data before training was a good idea, but it was too late for us since we noticed that after we had trained a lot of models with unbalanced data, and for training all the models again will takes a lot of time. For sure in the next projects we are going to balance the data from the begining.
​
This project allowed us to apply the theoretical knowledge we have learned to deal with practical problems and deal with real data. Compared with simulated data, we need to try more methods for this kind of real data and do more parameter learning to achieve a good effect. So for real data, especially massive data, it takes a long time to select and optimize our model.
​
Finally we ended with an accuracy of 94%.